In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_0 = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer_0 = AutoTokenizer.from_pretrained("gpt2")

model_1 = AutoModelForCausalLM.from_pretrained("Hiran03/GPT2finetuned")
tokenizer_1 = AutoTokenizer.from_pretrained("Hiran03/GPT2finetuned")
    

# # Input query
# query = "what is microstructure"

# # Tokenize the input


# print("Pretrained :")

# inputs = tokenizer_0(query, return_tensors="pt")
# # Generate response
# output_tokens = model_0.generate(
#     inputs["input_ids"],
#     attention_mask=inputs["attention_mask"],
#     pad_token_id=model_0.config.eos_token_id,
#     max_length = 100,
# )
# response = tokenizer_0.decode(output_tokens[0], skip_special_tokens=True)
# print(response)

# print ("\n \n")
# print("Fine-tuned : ")

# inputs = tokenizer_1(query, return_tensors="pt")
# # Generate response
# output_tokens = model_1.generate(
#     inputs["input_ids"],
#     attention_mask=inputs["attention_mask"],
#     pad_token_id=model_1.config.eos_token_id,
#     max_length = 100,
# )
# response = tokenizer_1.decode(output_tokens[0], skip_special_tokens=True)
# print(response)


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [2]:
# Push to Hugging Face Hub
model_name = "GPT2finetuned"
# model_1.push_to_hub(model_name)
# tokenizer_1.push_to_hub(model_name)

adapter_model.safetensors:   0%|          | 0.00/298k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\hiran\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hiran\.cache\huggingface\hub\models--Hiran03--GPT2finetuned. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/Hiran03/GPT2finetuned/commit/d87db242afba1331bc82b9597cb7a8fec6d3e0ec', commit_message='Upload tokenizer', commit_description='', oid='d87db242afba1331bc82b9597cb7a8fec6d3e0ec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Hiran03/GPT2finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Hiran03/GPT2finetuned'), pr_revision=None, pr_num=None)

In [ ]:
import os
from PyPDF2 import PdfReader

# Path to the folder containing PDF files
pdf_folder = r"papers"

text = ""
# Iterate over all files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):  # Process only PDF files
        pdf_path = os.path.join(pdf_folder, filename)
        print(f"Processing file: {filename}")
        
        # Read the PDF
        reader = PdfReader(pdf_path)
        
        # Extract text from all pages
        for page in reader.pages:
            text += page.extract_text()
        

with open(os.path.join(os.getcwd(),"rag_text.txt"), "w", encoding="utf-8") as f:
    f.write(text)
import text_preprocess as text_preprocess
text_preprocess.preprocessing(os.path.join(os.getcwd(),"rag_text.txt"))



Processing file: paper_1.pdf
Processing file: paper_2.pdf


In [22]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores.utils import DistanceStrategy
from sentence_transformers import SentenceTransformer, models

# Step 1: Load the Text File
file_path = "rag_text.txt"  # Replace with your text file path
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Step 2: Split the Text into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", ",", " "],
    chunk_size=900,  # Maximum chunk size
    chunk_overlap=200  # Overlap between chunks
)
chunks = text_splitter.split_text(text)

# Convert each chunk into a LangChain Document
docs_processed = [Document(page_content=chunk) for chunk in chunks]

# Step 3: Set Up Sentence Transformer
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # Change to 'cuda' if GPU is available
    encode_kwargs={"normalize_embeddings": True},  # Normalize embeddings for cosine similarity
)

print("Embedding model successfully initialized!")
# Step 4: Create FAISS Vector Store
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed,
    embedding_model,
    distance_strategy=DistanceStrategy.COSINE
)

# Step 5: Save the FAISS Index
faiss_file_path = "faiss_index"
KNOWLEDGE_VECTOR_DATABASE.save_local(faiss_file_path)
print(f"FAISS index saved at: {faiss_file_path}")


Embedding model successfully initialized!
FAISS index saved at: faiss_index


In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# Step 1: Load the GPT-2 model and tokenizer
model_name = "Hiran03/GPT2finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 2: Create a Hugging Face pipeline
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=100)

# Step 3: Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Step 4: Generate text
prompt = "Once upon a time in a distant land,"
output = llm(prompt)
print(output)


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time in a distant land, he might have looked like he knew what he was getting into! However, here, he had a face which was as cold as ice and dark as the sky beneath his eyes.

But what he saw was so frightening as to leave a very frightened glance on the girl's face! Even with his knowledge of the magic sword, the demon was a very small man! What kind of magic might she be and would she give him a magic that could


In [69]:
import numpy as np
from langchain.chains import RetrievalQA

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# Step 1: Load the Text File
file_path = "rag_text.txt"  # Replace with your text file path
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Step 2: Split the Text into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", ",", " "],
    chunk_size=1000,  # Maximum chunk size
    chunk_overlap=200  # Overlap between chunks
)
chunks = text_splitter.split_text(text)

# Step 3: Set Up Sentence Transformer
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # Change to 'cuda' if GPU is available
    encode_kwargs={"normalize_embeddings": True},  # Normalize embeddings for cosine similarity
)

query = "what is crystallinity"
# Step 1: Load the GPT-2 model and tokenizer
model_name = "Hiran03/GPT2finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 2: Create a Hugging Face pipeline
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=1024, truncation = True)

# Step 3: Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)
vectorstore = FAISS.from_texts(chunks,embedding_model)
chain = RetrievalQA.from_llm(llm=llm, retriever=vectorstore.as_retriever())
result = chain(query)





Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [70]:
result

{'query': 'what is crystallinity',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext:\naqueous solution of nh46mo7o244h2o precursor6 however these\nmethods have limitations in growing largesize single crystals with\nrapid growth rates now as they report in nature materials he jiang\nand lleagues have effectively utilized the wellknown cz method\nfor the growth of 2d tmd particularly mos2 ref 7 they achieved\nthe production of single crystals exceeding 15 cm in size with an\nimpressive growth rate of 75 m s1 marking a clear advancement\nover previous techniques\nthe key aspect of the 2d cz method is its use of a liquidliquid\ninterface spreading mechanism to suppress nucleation and enhance\ncrystal growth rate this enables the growth of centimetrescale\nsingle crystals with low defect density moreover the weak adhesion\nbetween the mos2 crystal and the 